The finite element method requires computing certain integrals, for example, in the right-hand side, there are integrals of the form
$$
    \int f(x) \phi_i(x) \, dx
$$
and in the stiffness matrix, there are integrals of the form
$$
    \int \phi_i'(x) \phi_j'(x) \, dx.
$$
In these expression expression, $\phi_i$ represents a basis function.
We have seen piecewise linear basis functions; more generally, we can consider piecewise polynomial basis functions.
Simple closed-form expressions for integrals of these functions (and their derivatives) exist.
However, $f$ may in principle be arbitrary.
It can even be considered to be a "black box": for example, $f$ may come from data or from empirical observations.
Using exact formulas for integrals involving $f$ is clearly not practical.

Therefore, we compute (or approximate) the integrals in the finite element method using **numerical quadrature**.
A quadrature formula consists of points $\{ x_i \}$ (also called the abscissas) and weights $\{ w_i \}$.
Then, the integral of a function $F(x)$ is approximated
$$
    \int_{-1}^1 F(x) \, dx \approx \sum_{i=1}^N F(x_i) w_i.
$$
A simple transformation lets us use quadrature to approximate integrals over any interval $[a,b]$.

Let $T(x) = \frac{1}{2}(b-a)(x + 1) + a$.
Note that $T : [-1, 1] \mapsto [a,b]$.
Also, $T'(x) = \frac{1}{2}(b-a)$.

Note then that, by change of variables,
$$
    \int_a^b F(x) \, dx = \int_{-1}^1 F(T(x)) T'(x) \, dx \approx \frac{1}{2}(b-a) \sum_{i=1}^N F(T(x_i)) w_i.
$$

Since the quadrature formula is an approximation, we can talk about its order of accuracy.
Let $Q(F,a,b)$ denote the quadrature formula above,
$$
    Q(F,a,b) = \frac{1}{2}(b-a) \sum_{i=1}^N F(T(x_i)) w_i.
$$
If $h = b - a$, then we expect the quadrature error
$$
    \left| \int_a^b F(x)\,dx - Q(F,a,b) \right| \to 0 \quad\text{as}\quad h \to 0
$$
In particular, in many cases, we have $\left| \int_a^b F(x)\,dx - Q(F,a,b) \right| = \mathcal{O}(h^\alpha)$ for some $\alpha$.
Taylor series arguments can be used to find $\alpha$.

If we have a **fixed** interval $[a,b]$, and we want to get increasingly accurate approximations to $\int_a^b F(x) \, dx$ using a quadrature formula, we can subdivide the interval into $M$ subintervals, each of length $h = (b-a) / M$.
Then, on each subinterval, the error will scale like $h^{\alpha}$, and so the total error over all intervals will scale like
$$
    M h^\alpha = M ((b-a) / M)^\alpha = (b-a)^\alpha / M^{\alpha - 1} = (b-a) h^{\alpha - 1}.
$$

For most common quadrature formulas, $Q(F,a,b)$ will be **exact** if $F$ is a polynomial of degree $p$ or less (the specific $p$ will depend on the formula).
This implies that the quadrature error on an interval of length $h$ will scale like $h^{p + 2}$,(why?) and so, using quadrature and subdivision as described above, the error over a fixed interval will scale like $h^{p+1}$.

The most commonly used quadratures are known as **Gaussian quadratures**.
These are quadrature formulas that maximize the **degree of exactness** in terms of polynomial degree.
Note that if $n$ points are chosen arbitrarily, then we can come up with a quadrature formula that will integrate polynomials of degree $n - 1$ exactly. (Why?)
However, **Gauss-Legendre** formula integrates polynomials of degree $2n - 1$ exactly.
The abscissas all lie in the interior of the interval.

There are also **Gauss-Radau** and **Gauss-Lobatto** quadratures.
Radau quadrature includes one endpoint, and has degree of exactness $2n - 2$.
Lobatto quadrature includes both endpoints, and has degree of exactness $2n - 3$.

These formulas are closely related to **orthogonal polynomials** and there is a deep and interesting theory of quadrature formulas.
For our purposes, we can look up tables of these formulas in textbooks or on Wikipedia.

In [1]:
import numpy as np

In [2]:
def quad_1d(f, a, b):
    # Quadrature points in the reference internal
    xq_1 = -1/np.sqrt(3)
    xq_2 = 1/np.sqrt(3)

    h = b - a
    x_1 = h*0.5*(xq_1 + 1) + a
    x_2 = h*0.5*(xq_2 + 1) + a
    return 0.5*h*(f(x_1) + f(x_2))

In [3]:
def f(x):
    return np.cos(x) * np.exp(np.sin(x))

def int_f(x):
    # return np.tan(x) - x
    return np.exp(np.sin(x))

a = 1

print("h               Error         Rate")
print("=" * 35)

h_prev = 0
error_prev = 0

for i in range(14):
    h = 2 ** -i
    b = a + h
    v_exact = int_f(b) - int_f(a)
    v_approx = quad_1d(f, a, b)
    error = np.abs(v_exact - v_approx)
    if h_prev != 0:
        rate = "{: .2f}".format(np.log(error / error_prev) / np.log(h / h_prev))
    else:
        rate = " ----"
    print("{:.10f}    {:.3e}    {}".format(h, error, rate))
    error_prev = error
    h_prev = h


h               Error         Rate
1.0000000000    1.213e-03     ----
0.5000000000    1.537e-04     2.98
0.2500000000    5.550e-06     4.79
0.1250000000    1.741e-07     4.99
0.0625000000    5.368e-09     5.02
0.0312500000    1.660e-10     5.02
0.0156250000    5.156e-12     5.01
0.0078125000    1.610e-13     5.00
0.0039062500    5.190e-15     4.95
0.0019531250    2.424e-16     4.42
0.0009765625    1.115e-16     1.12
0.0004882812    3.747e-16    -1.75
0.0002441406    1.821e-16     1.04
0.0001220703    3.629e-17     2.33


_Note:_ the errors decrease at the expected rate, until they reach a level of around $10^{-16}$, at which point round-off error dominates, and the error stagnates.

In [4]:
a = 1
b = 2

print("h               Error         Rate")
print("=" * 35)

h_prev = 0
error_prev = 0

for i in range(16):
    M = 2 ** i
    h = (b - a) / M

    v_exact = int_f(b) - int_f(a)
    v_approx = 0.0

    a_j = a
    for j in range(M):
        b_j = a_j + h
        v_approx += quad_1d(f, a_j, b_j)
        a_j = b_j

    error = np.abs(v_exact - v_approx)
    if h_prev != 0:
        rate = "{: .2f}".format(np.log(error / error_prev) / np.log(h / h_prev))
    else:
        rate = " ----"
    print("{:.10f}    {:.3e}    {}".format(h, error, rate))
    error_prev = error
    h_prev = h

h               Error         Rate
1.0000000000    1.213e-03     ----
0.5000000000    5.438e-05     4.48
0.2500000000    3.202e-06     4.09
0.1250000000    1.973e-07     4.02
0.0625000000    1.229e-08     4.01
0.0312500000    7.674e-10     4.00
0.0156250000    4.795e-11     4.00
0.0078125000    2.997e-12     4.00
0.0039062500    1.877e-13     4.00
0.0019531250    1.163e-14     4.01
0.0009765625    6.106e-16     4.25
0.0004882812    2.498e-16     1.29
0.0002441406    1.277e-15    -2.35
0.0001220703    1.943e-16     2.72
0.0000610352    2.276e-15    -3.55
0.0000305176    4.996e-16     2.19
